# Table of Contents
 <p><div class="lev1"><a href="#Multiband-Resting-State-fMRI-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Multiband Resting-State fMRI</a></div><div class="lev2"><a href="#Summary-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Summary</a></div><div class="lev2"><a href="#Methodology-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Methodology</a></div><div class="lev1"><a href="#Measure-Dynamic-Functional-Connectivity-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Measure Dynamic Functional Connectivity</a></div><div class="lev2"><a href="#Initialize-Environment-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2"><a href="#Generate-List-of-Data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2"><a href="#Design-and-Run-DyNe-Pipeline-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Design and Run DyNe Pipeline</a></div><div class="lev3"><a href="#Generate-Pipe-Definitions-(JSON-Files)-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Generate Pipe Definitions (JSON Files)</a></div><div class="lev3"><a href="#Generate-Pipeline-Definitions-(JSON-File)-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Generate Pipeline Definitions (JSON File)</a></div><div class="lev3"><a href="#Run-the-Pipeline-(in-Parallel)-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Run the Pipeline (in Parallel)</a></div><div class="lev2"><a href="#Plot-Adjacency-Matrices-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Plot Adjacency Matrices</a></div>

# Multiband Resting-State fMRI

## Summary

*__Introduction__*

  * The brain is a complex organ that manifests a network of interactions, at the finer scale, between neurons and neuron populations, and at the meso-scale, between large functional domains. Over time, the pattern of interactions between brain regions change, describing a dynamic system capable of supporting cognition during tasks and at rest.
  
  * Recent advances in functional brain imaging make it possible to acquire images from multiple brain slices simultaneously, known as multiband imaging. 
  
  * Multiband imaging improves the sampling frequency of the blood oxygenation level dependant (BOLD) signal from $\approx$0.5 Hz to $\approx$2.0 Hz. With faster sampling of the BOLD signal, we are able to examine functional interactions across whole brain at unprecendent temporal resolution.
  
  * Previous work has indicated that functional associations between low-frequency components of the fMRI signal (0–0.15 Hz) can be attributed to task-related functional connectivity, whereas associations between high-frequency components (0.2–0.4 Hz) cannot. This frequency specificity of task-relevant functional connectivity is likely to be due at least in part to the hemodynamic response function, which might act as a noninvertible bandpass filter on underlying neural activity.
  
*__References__*
  - Pan, Wen-Ju, et al. "Infraslow LFP correlates to resting-state fMRI BOLD signals." Neuroimage 74 (2013): 288-297.
  - F. T. Sun, L. M. Miller, and M. D’Esposito, “Measuring interregional functional connectivity using coherence and partial coherence analyses of fMRI data.” Neuroimage 21 (2004): 647–658.



## Methodology

*__Multi-Taper Coherence__*

  * Multi-Taper methods are designed to limit the leakage of spectral power from other frequency bands into the frequency bands of interest. This is a critical concern when spectral properties of the signal are computed in discrete temporal windows. Different shapes of the temporal window can cause unwanted effects in the Fourier domain.

  * The choice of window length and shape impacts the allowable frequency resolution of the signal power spectrum. Slepian sequences, or orthogonal, wave-like eigenfunctions, give the optimal windowing to achieve a spectral bandwidth resolution on the interval $[-W, W]$. To achieve this resolution, multiple sequences are required, such that the optimal number of Slepian sequences, $k\approx2WT$, where $k$ is the number of Slepian sequences, $2W$ is the desired spectral resolution bandwidth, and $T$ is the duration of the window.
  
  * Suppose we wanted to estimate dynamic functional connectivity of fMRI in 30 second windows between 0.034 Hz and 0.167 Hz (i.e. a bandwidth resolution of 0.133 Hz), then the time-bandwidth product would be $4.0 = 0.133 Hz * 30 sec$, and the optimal number of tapers would be $2*4.0-1 = 7$.
  
   

# Measure Dynamic Functional Connectivity

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    
except:
    print 'NOT IPYTHON'

from __future__ import division
from IPython.display import display

import os
import sys
import glob
import json

import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

import dyne

path_CoreData = '/Users/akhambhati/Remotes/hoth_research/CoreData/fMRI_multiband-mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/hoth_research/PeriphData/ds-NMF_Subnetworks'
path_ExpData = path_PeriphData + '/e01-Dyne_FuncNetw'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Generate List of Data

In [ ]:
subjs = [full_subj_path.split('/')[-1]
         for full_subj_path in glob.iglob('{}/Subjects/*'.format(path_CoreData))]

subj_ids = {}
proc_path = []
for subj in subjs:
    run_path = '{}/Subjects/{}/*'.format(path_CoreData, subj)

    subj_ids[subj] = []
    run_ids = {}
    for full_run_path in glob.iglob(run_path):
        subj_ids[subj].append(full_run_path.split('/')[-1])

## Design and Run DyNe Pipeline

### Generate Pipe Definitions (JSON Files)

In [ ]:
for subj, dates in subj_ids.items():
    for date_id in dates:
        
        # Generate path to the input fMRI clip
        proc_item = '{}/Subjects/{}/{}/HOA112.csv'.format(path_CoreData,
                                                          subj, date_id)
        if not os.path.exists(proc_item):
            raise IOError('Procesing item: {}, not found'.format(proc_item))
        
        # Define the pipe definitions
        pipe_defs = {
            "SOURCE": {
                "PIPE_NAME": "Raw_fMRI",
                "PIPE_MODULE": "dyne.interface.offline",
                "PIPE_CLASS": "CSVSignal",
                "PIPE_PARAM": {
                    "signal_path": proc_item,
                    "sample_frequency": 2.0,
                    "win_len": 30.0,
                    "win_disp": 30.0
                }
            },

            "FLOW": [
                {
                    "PIPE_NAME": "MTCoh",
                    "PIPE_MODULE": "dyne.adjacency.coherence",
                    "PIPE_CLASS": "MTCoh",
                    "PIPE_PARAM": {
                        "time_band": 2.5,
                        "n_taper": 4,
                        "cf": [0.034, 0.20]
                    }
                },

                {
                    "PIPE_NAME": "CachePipeline",
                    "PIPE_MODULE": "dyne.logger.cache",
                    "PIPE_CLASS": "SaveHDF",
                    "PIPE_PARAM": {
                        "path": "{}/{}.{}.dyne_output.hdf".format(path_ExpData,
                                                                  subj, 
                                                                  date_id)
                    }
                }
            ],

            "LOG": {
                "PATH": "{}/{}.{}.dyne_log.csv".format(path_ExpData,
                                                       subj, 
                                                       date_id)
            }
        }
        json.dump(pipe_defs, open("{}/{}.{}.dyne_pdef.json".format(path_ExpData,
                                                                   subj,
                                                                   date_id),
                                  'w'))

### Generate Pipeline Definitions (JSON File)

In [ ]:
pipeline_def = {
    "Raw_fMRI": [
        "MTCoh"],

    "MTCoh": [
        "CachePipeline"],
    
    "CachePipeline": [
        "None"]
}
json.dump(pipeline_def, open("{}/fmri_coh_pipeline.json".format(path_ExpData),
                             "w"))

### Run the Pipeline (in Parallel) 
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/*.dyne_output.hdf".format(path_ExpData))
rm_logs = glob.glob("{}/*.dyne_log.csv".format(path_ExpData))

for rm_type in [rm_outp, rm_logs]:
    for path in rm_type:
        os.remove(path)

In [ ]:
from multiprocessing import Pool
parallel_run = False

# Generate list of all pipe_defs and the pipeline architecture
proc_pdef = glob.glob("{}/*.*.dyne_pdef.json".format(path_ExpData))
proc_pline = "{}/fmri_coh_pipeline.json".format(path_ExpData)

# Setup helper function to map pipeline run
def _start_helper(path):
    print('\nProcessing: {}'.format(path))
    pline = dyne.pipeline.Pipeline(pipe_defs_json=path,
                                   pipeline_def_json=proc_pline)
    pline.start_pipeline()
    

if parallel_run:
    mp = Pool(6)
    mp.map(_start_helper, proc_pdef)
else:
    map(_start_helper, proc_pdef)

## Plot Adjacency Matrices

In [ ]:
# Set Parameters
n_adj = 200
n_row = int(np.sqrt(n_adj))
n_col = int(np.ceil(n_adj / n_row))

# Get list of DyNe Output Files
dyne_outp = glob.glob("{}/*.dyne_output.hdf".format(path_ExpData))
dyne_logs = glob.glob("{}/*.dyne_log.csv".format(path_ExpData))

# Figure
%matplotlib inline
plt.figure(figsize=(16, 12))
for ii in xrange(n_adj):
    outp, log = np.random.permutation(zip(dyne_outp, dyne_logs))[0]
    ix = np.random.randint(0, 20)
    
    df_log = pd.read_csv(log, delimiter=',')
    pipe_hash = df_log[df_log.PIPE_NAME == 'MTCoh'].DOWNSTREAM_HASH[0]
    
    df_outp = h5py.File(outp, 'r')
    data = df_outp[pipe_hash]['data'][ix, :, :]
    df_outp.close()

    ax = plt.subplot(n_row, n_col, ii+1)
    ax.matshow(data)
    ax.set_axis_off()